# Supermarket EDA

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
mon = pd.read_csv('data/monday.csv', sep=';', index_col=0, parse_dates=True)
tue = pd.read_csv('data/tuesday.csv', sep=';', index_col=0, parse_dates=True)
wed = pd.read_csv('data/wednesday.csv', sep=';', index_col=0, parse_dates=True)
thu = pd.read_csv('data/thursday.csv', sep=';', index_col=0, parse_dates=True)
fri = pd.read_csv('data/friday.csv', sep=';', index_col=0, parse_dates=True)


In [3]:
def new_cust_col(a):
    '''updated the customer columns of a dataframe adding the weekly day'''
    lst = ['m_','tu_','w_','th_','fr_']
    wd = a.index[1].weekday()
    a['cust_no'] = lst[wd]+a['customer_no'].astype('str')
    a.drop('customer_no', axis=1, inplace=True)
    a.reset_index(inplace=True)
    return a

In [4]:
mon = new_cust_col(mon)
tue = new_cust_col(tue)
wed = new_cust_col(wed)
thu = new_cust_col(thu)
fri = new_cust_col(fri)

In [5]:
mon

,timestamp,location,cust_no
0,2019-09-02 07:03:00,dairy,m_1
1,2019-09-02 07:03:00,dairy,m_2
2,2019-09-02 07:04:00,dairy,m_3
3,2019-09-02 07:04:00,dairy,m_4
4,2019-09-02 07:04:00,spices,m_5
...,...,...,...
4879,2019-09-02 21:49:00,checkout,m_1442
4880,2019-09-02 21:49:00,checkout,m_1444
4881,2019-09-02 21:49:00,dairy,m_1445
4882,2019-09-02 21:50:00,dairy,m_1446


In [6]:
new_mon = mon.groupby(['timestamp','cust_no'])['location'].value_counts().unstack()
new_mon.fillna(0,inplace=True)
new_mon

location                     checkout  dairy  drinks  fruit  spices
timestamp           cust_no                                        
2019-09-02 07:03:00 m_1           0.0    1.0     0.0    0.0     0.0
                    m_2           0.0    1.0     0.0    0.0     0.0
2019-09-02 07:04:00 m_3           0.0    1.0     0.0    0.0     0.0
                    m_4           0.0    1.0     0.0    0.0     0.0
                    m_5           0.0    0.0     0.0    0.0     1.0
...                               ...    ...     ...    ...     ...
2019-09-02 21:49:00 m_1442        1.0    0.0     0.0    0.0     0.0
                    m_1444        1.0    0.0     0.0    0.0     0.0
                    m_1445        0.0    1.0     0.0    0.0     0.0
2019-09-02 21:50:00 m_1446        0.0    1.0     0.0    0.0     0.0
                    m_1447        0.0    0.0     0.0    1.0     0.0

[4884 rows x 5 columns]

In [7]:
ciao = pd.DataFrame(data=new_mon.index.get_level_values(1).value_counts() == 1)
lst = list(ciao[ciao['cust_no']==True].index)
lst

['m_1446', 'm_1439', 'm_1447', 'm_1445']

In [8]:
new_mon.reset_index(inplace=True)
new_mon.index.name = 'index'
new_mon

location,timestamp,cust_no,checkout,dairy,drinks,fruit,spices
index,,,,,,,
0,2019-09-02 07:03:00,m_1,0.0,1.0,0.0,0.0,0.0
1,2019-09-02 07:03:00,m_2,0.0,1.0,0.0,0.0,0.0
2,2019-09-02 07:04:00,m_3,0.0,1.0,0.0,0.0,0.0
3,2019-09-02 07:04:00,m_4,0.0,1.0,0.0,0.0,0.0
4,2019-09-02 07:04:00,m_5,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...
4879,2019-09-02 21:49:00,m_1442,1.0,0.0,0.0,0.0,0.0
4880,2019-09-02 21:49:00,m_1444,1.0,0.0,0.0,0.0,0.0
4881,2019-09-02 21:49:00,m_1445,0.0,1.0,0.0,0.0,0.0


In [ ]:
pd.DataFrame(data=[new_mon['timestamp'].max()+1,])

In [12]:
new_mon['timestamp'].max()

Timestamp('2019-09-02 21:50:00')

In [ ]:
###### need to: add a minute to the timestamp, then write a loop for creating data frame with columns: that timestamp, one of the cust_no
#in the list, 1.0,0.0,0.0,0.0,0.0
#append the dataframes to new mon. 
#do the same for tue, wed, ect.
#Concat the whole